# Global Warming

This notebook builds on the Copernicus Climate Change Service (C3S) global temperature trend monitor and provides monthly updates on global air temperatures based on the latest **ERA5 reanalysis data**. By analyzing the 30-year linear trend up to the most recent data, it estimates the current level of global warming compared to pre-industrial temperatures and predicts when the 1.5°C threshold set by the Paris Agreement might be reached.

This Jupyter notebook contains all the Python code used to calculate the statistics and create the visualizations for the global temperature trend monitor.

Datasets
* ERA5 reanalysis data
* ECV (Essential Climate Variables)
* IPCC Projection

## Codes

In [1]:
import os

''' ---- Specify Directories Here ---- '''
working_folder = r".\data\global_warming"
era5_folder = r".\data\global_warming\era5"
ecv_folder = r".\data\global_warming\ecv"
ipcc_folder = r".\data\global_warming\ipcc"
''' ----- End of Declaration ---- '''

os.makedirs(working_folder, exist_ok=True)
os.makedirs(era5_folder, exist_ok=True)
os.makedirs(ecv_folder, exist_ok=True)
os.makedirs(ipcc_folder, exist_ok=True)

In [2]:
import cdsapi

def main():
    # API key for authentication
    api_key = "fdae60fd-35d4-436f-825c-c63fedab94a4"
    api_url = "https://cds.climate.copernicus.eu/api"

    # Creation of the CDS API client
    client = cdsapi.Client(url=api_url, key=api_key)
    return client

## ERA5

In [3]:
dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": ["2m_temperature"],
    "year": [str(year) for year in range(1979, 2025)],
    "month": [str(month).zfill(2) for month in range(1, 13)],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "unarchived",
    "area": [90, -180, -90, 180],
}

In [4]:
# Execute it to download the dataset:
def main_retrieve():
    dataset_filename = f"{dataset}-global.nc"
    dataset_filepath = os.path.join(era5_folder, dataset_filename)

    # Download the dataset only if the dataset has not been downloaded before
    if not os.path.isfile(dataset_filepath):
        # Call the CDS client only if the dataset has not been downloaded before
        client = main()
        # Download the dataset with the defined request parameters
        client.retrieve(dataset, request, dataset_filepath)
    else:
        print("Dataset already downloaded.")

if __name__ == "__main__":
    main_retrieve()

Dataset already downloaded.


In [5]:
import netCDF4 as nc

dataset_filename = f"{dataset}-global.nc"
nc_filepath = os.path.join(era5_folder, dataset_filename)

nc_dataset = nc.Dataset(nc_filepath, mode='r')
# List all variables in the dataset
variables_list = nc_dataset.variables.keys()
print(f"Available variables: {list(variables_list)}")

Available variables: ['number', 'valid_time', 'latitude', 'longitude', 'expver', 't2m']


## ECV

In [6]:
# Note: Dataset available in grib format ("data_format": "netcdf" doesnt work)

dataset = "ecv-for-climate-change"
request = {
    "variable": ["surface_air_temperature"],
    "origin": ["era5"],
    "product_type": ["climatology"],
    "climate_reference_period": ["1981_2010"],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
}

In [7]:
# Execute it to download the dataset:
def main_retrieve():
    dataset_filename = f"{dataset}-global.zip"
    dataset_filepath = os.path.join(ecv_folder, dataset_filename)

    # Download the dataset only if the dataset has not been downloaded before
    if not os.path.isfile(dataset_filepath):
        # Call the CDS client only if the dataset has not been downloaded before
        client = main()
        # Download the dataset with the defined request parameters
        client.retrieve(dataset, request, dataset_filepath)
    else:
        print("Dataset already downloaded.")

if __name__ == "__main__":
    main_retrieve()

Dataset already downloaded.


In [8]:
import netCDF4 as nc
import zipfile

dataset_filename = f"{dataset}-global_nc.zip"
zip_filepath = os.path.join(ecv_folder, dataset_filename)

with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
    zip_ref.extractall(ecv_folder)

    

FileNotFoundError: [Errno 2] No such file or directory: '.\\data\\global_warming\\ecv\\ecv-for-climate-change-global_nc.zip'

## IPCC

In [21]:
import requests
import pandas as pd

# URL der CSV-Daten
IPCC_DATA = "https://apps.climate.copernicus.eu/global-temperature-trend-monitor/ipcc-projections-sr15.csv"

# Funktion zum Herunterladen einer Datei mit Zertifikatsüberprüfung
def download_file(url, filename):
    if os.path.exists(filename):
        print(f"Datei {filename} ist bereits vorhanden. Kein Download erforderlich.")
        return
    try:
        response = requests.get(url, verify=True)
        response.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.SSLError:
        response = requests.get(url, verify=False)  # Falls Zertifikat fehlt, ohne Zertifikatsüberprüfung herunterladen
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"Datei {filename} erfolgreich heruntergeladen.")

# Dateiname und Ordner festlegen
filename = "ipcc_projections_sr15.csv"
ipcc_download = os.path.join(ipcc_folder, filename)

# Datei herunterladen
download_file(IPCC_DATA, ipcc_download)

# CSV-Datei laden
ipcc = pd.read_csv(ipcc_download)

print(ipcc)

Datei .\data\global_warming\ipcc\ipcc_projections_sr15.csv ist bereits vorhanden. Kein Download erforderlich.
     year  min_t  max_t
0    1850 -0.023 -0.035
1    1851 -0.022 -0.033
2    1852 -0.021 -0.032
3    1853 -0.020 -0.030
4    1854 -0.019 -0.029
..    ...    ...    ...
246  2096  0.924  2.011
247  2097  0.922  2.011
248  2098  0.921  2.011
249  2099  0.920  2.010
250  2100  0.918  2.010

[251 rows x 3 columns]
